<a href="https://colab.research.google.com/github/hotsun1508/Session1/blob/master/%5B%ED%81%AC%EB%A1%A4%EB%A7%81%EC%84%B8%EC%85%98%5DYoutube_Rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Youtube Rank에서 인기 동영상 크롤링하기 🤗

In [3]:
# 라이브러리 설치
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 968 kB 4.3 MB/s 
     |████████████████████████████████| 359 kB 41.2 MB/s 
     |████████████████████████████████| 138 kB 55.5 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 4.0 MB 41.7 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-

### Youtube Rank 인기 콘텐츠 한 페이지 크롤링 => top 100

1. chrome driver 열기
2. 페이지 접속하기
3. BeautifulSoup 활용하여 html형태로 불러온 데이터를 정리하기 
4. 원하는 정보를 찾기 
    - 채널 정보 뭉치별로 top 100 찾기
5. Excel 파일로 데이터 저장 (list -> DataFrame)
6. 시간되면 추가 기능 

In [4]:
# 1. import 라이브러리
from selenium import webdriver
from bs4 import BeautifulSoup

import time    #  time.sleep(대기시간)
import pandas as pd   

In [5]:
## Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

In [6]:
# 2. 페이지 접속하기
url = "https://youtube-rank.com/board/bbs/board.php?bo_table=youtube" 
driver.get(url)

In [7]:
# 3. BeautifulSoup 활용하여 데이터 정리하기

html = driver.page_source 
html

'<html lang="ko"><head>\n    <meta charset="utf-8">\n    <meta http-equiv="imagetoolbar" content="no">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <title>유튜브 랭킹 1 페이지</title>\n    <link rel="stylesheet" href="/css/default.css?ver=2022022">\n<link rel="stylesheet" href="/css/bootstrap.css?ver=2022022">\n<link rel="stylesheet" href="/css/nifty.min.css?ver=2022022">\n<link rel="stylesheet" href="/css/premium/icon-sets/icons/line-icons/premium-line-icons.min.css?ver=2022022">\n<link rel="stylesheet" href="/css/themes/type-c/theme-well-red.css?ver=2022022">\n<link rel="stylesheet" href="https://youtube-rank.com/board/theme/youtube/skin/outlogin/youtuberank/style.css?ver=2022022">\n<link rel="stylesheet" href="https://youtube-rank.com/board/skin/board/sns_channel/list.skin.css?2022022">\n<link rel="stylesheet" href="https://unpkg.com/aos@2.3.1/dist/aos.css">\n    <link rel="apple-touch-icon" sizes="180x180" href="/img/apple-touch-icon.png">\n    <link rel="shortcut icon" href

In [8]:
soup = BeautifulSoup(html, 'html.parser')
soup

<html lang="ko"><head>
<meta charset="utf-8"/>
<meta content="no" http-equiv="imagetoolbar"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>유튜브 랭킹 1 페이지</title>
<link href="/css/default.css?ver=2022022" rel="stylesheet"/>
<link href="/css/bootstrap.css?ver=2022022" rel="stylesheet"/>
<link href="/css/nifty.min.css?ver=2022022" rel="stylesheet"/>
<link href="/css/premium/icon-sets/icons/line-icons/premium-line-icons.min.css?ver=2022022" rel="stylesheet"/>
<link href="/css/themes/type-c/theme-well-red.css?ver=2022022" rel="stylesheet"/>
<link href="https://youtube-rank.com/board/theme/youtube/skin/outlogin/youtuberank/style.css?ver=2022022" rel="stylesheet"/>
<link href="https://youtube-rank.com/board/skin/board/sns_channel/list.skin.css?2022022" rel="stylesheet"/>
<link href="https://unpkg.com/aos@2.3.1/dist/aos.css" rel="stylesheet"/>
<link href="/img/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicon.ico" rel="shortcut icon" type="image

# 4. 원하는 정보 찾기 
```
soup.select('태그명')
soup.select('.class속성값')
soup.select('부모태그정보 > 자식태그정보')
soup.select('#id속성값')
```

### Quiz) 채널 html 뭉치 찾아보기

In [9]:
channel_list = soup.select('tr')
len(channel_list)

102

In [10]:
channel_list = soup.select('tbody > tr')
len(channel_list)

101

In [11]:
channel_list = soup.select('table > tbody > tr')
len(channel_list)

101

In [12]:
channel_list = soup.select('form > table > tbody > tr')
len(channel_list)

100

### hint: class명으로 찾기 

In [13]:
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [14]:
# 찾은 데이터들을 하나씩 살펴봅시다.
channel_list[-1]

<tr class="aos-init" data-aos="fade-up" data-aos-duration="800">
<td class="rank">
                        100                    </td>
<td class="td_img">
<div class="info_img"><a href="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&amp;wr_id=3202"><img class="lazyload" data-src="https://yt3.ggpht.com/2fI3tsk5YcVrXdyrK9kJTl0EvQs0oN6PI6a-lLaNtCb94bcAvRAYbXpu1fFv4yzfxG9vP3j2yw=s88-c-k-c0x00ffffff-no-rj" height="88" src="/img/img_bg.png" width="88"/></a></div>
<p class="info_rank">100</p>
</td>
<td class="subject">
<h1>
<p <a="" class="category" href="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&amp;sca=TV%2F%EB%B0%A9%EC%86%A1">[TV/방송]
                                
                                </p>
<a href="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&amp;wr_id=3202">
								
								채널 십오야							</a>
<span>
<i class="fa fa-comment"></i>
								0							</span>
<i aria-hidden="true" class="fa fa-heart"></i> </h1>
<h2><span><a href="

In [15]:
# 채널 100개 -> 1개 선택 
channel = channel_list[0]
channel

<tr class="aos-init aos-animate" data-aos="fade-up" data-aos-duration="800">
<td class="rank">
                        1                    </td>
<td class="td_img">
<div class="info_img"><a href="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&amp;wr_id=3203"><img class="lazyload" data-src="https://yt3.ggpht.com/ytc/AKedOLTyoe1McP_IhmAA09N0JCVZCx_naUBUUQCs5F_0Ug=s88-c-k-c0x00ffffff-no-rj-mo" height="88" src="https://yt3.ggpht.com/ytc/AKedOLTyoe1McP_IhmAA09N0JCVZCx_naUBUUQCs5F_0Ug=s88-c-k-c0x00ffffff-no-rj-mo" width="88"/></a></div>
<p class="info_rank">1</p>
</td>
<td class="subject">
<h1>
<p <a="" class="category" href="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&amp;sca=%EC%9D%8C%EC%95%85%2F%EB%8C%84%EC%8A%A4%2F%EA%B0%80%EC%88%98">[음악/댄스/가수]
                                
                                </p>
<a href="https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&amp;wr_id=3203">
								
								BLACKPINK							</a>
<span>
<i class="

In [16]:
# 채널명 찾기
title = channel.select('h1 > a')
len(title)

1

### Quiz) soup.select() ↔ channel.select()

In [17]:
title = channel.select('h1 > a')[0].text.strip() # 공백 제거해줌 \t, \n 

title

'BLACKPINK'

In [18]:
## 카테고리 정보 정리해보기
category = channel.select('p.category')[0].text.strip() 
category

'[음악/댄스/가수]'

In [19]:
## 구독자 정보 정리해보기
subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
subscriber

'7390만'

In [20]:
## view 정보 정리해보기
view = channel.select('td.view_cnt')[0].text.strip() 
view

'235억0126만'

In [21]:
## 채널 하나의 정보를 수집하기 
# 1. 채널 선택하기
channel = channel_list[0]

# 2. 1개의 채널 정보를 정리
title = channel.select('h1 > a')[0].text.strip() 
category = channel.select('p.category')[0].text.strip() 
subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
view = channel.select('td.view_cnt')[0].text.strip() 

# 출력하기
print(title, category, subscriber, view)

BLACKPINK [음악/댄스/가수] 7390만 235억0126만


## 1개 채널의 정보 수집 -> 100개 여러개 채널 수집하기

In [22]:
## 채널 하나의 정보를 수집하기 
# 1. 채널 선택하기
channel = channel_list[0]  # 블랙핑크 

# 2. 1개의 채널 정보를 정리
title = channel.select('h1 > a')[0].text.strip() 
category = channel.select('p.category')[0].text.strip() 
subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
view = channel.select('td.view_cnt')[0].text.strip() 

# 출력하기
print(title, category, subscriber, view)
print("-------------------------------------------------")

## 채널 하나의 정보를 수집하기 
# 1. 채널 선택하기
channel = channel_list[1]  # 방탄 

# 2. 1개의 채널 정보를 정리
title = channel.select('h1 > a')[0].text.strip() 
category = channel.select('p.category')[0].text.strip() 
subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
view = channel.select('td.view_cnt')[0].text.strip() 

# 출력하기
print(title, category, subscriber, view)
print("-------------------------------------------------")

## 채널 하나의 정보를 수집하기 
# 1. 채널 선택하기
channel = channel_list[2]  # 하이브 

# 2. 1개의 채널 정보를 정리
title = channel.select('h1 > a')[0].text.strip() 
category = channel.select('p.category')[0].text.strip() 
subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
view = channel.select('td.view_cnt')[0].text.strip() 

# 출력하기
print(title, category, subscriber, view)


## 채널 하나의 정보를 수집하기 
# 1. 채널 선택하기
channel = channel_list[0]

# 2. 1개의 채널 정보를 정리
title = channel.select('h1 > a')[0].text.strip() 
category = channel.select('p.category')[0].text.strip() 
subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
view = channel.select('td.view_cnt')[0].text.strip() 

# 출력하기
print(title, category, subscriber, view)


BLACKPINK [음악/댄스/가수] 7390만 235억0126만
-------------------------------------------------
BANGTANTV [음악/댄스/가수] 6590만 159억7409만
-------------------------------------------------
HYBE LABELS [음악/댄스/가수] 6520만 223억8194만
BLACKPINK [음악/댄스/가수] 7390만 235억0126만


In [23]:
len(channel_list)

100

In [24]:
### 반복문 !!! 활용 -> 100개 정보 출력하기

for channel in channel_list: 

    title = channel.select('h1 > a')[0].text.strip() 
    category = channel.select('p.category')[0].text.strip() 
    subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
    view = channel.select('td.view_cnt')[0].text.strip() 

    # 출력하기
    print(title, category, subscriber, view)

BLACKPINK [음악/댄스/가수] 7390만 235억0126만
BANGTANTV [음악/댄스/가수] 6590만 159억7409만
HYBE LABELS [음악/댄스/가수] 6520만 223억8194만
SMTOWN [음악/댄스/가수] 3030만 244억0845만
Boram Tube Vlog [보람튜브 브이로그] [키즈/어린이] 2650만 110억5288만
1MILLION Dance Studio [음악/댄스/가수] 2520만 72억3740만
JYP Entertainment [음악/댄스/가수] 2440만 163억1461만
1theK (원더케이) [음악/댄스/가수] 2400만 221억3827만
Mnet K-POP [음악/댄스/가수] 1950만 148억8730만
KBS WORLD TV [TV/방송] 1780만 135억1256만
JFlaMusic [음악/댄스/가수] 1750만 36억0286만
Jane ASMR 제인 [음식/요리/레시피] 1640만 61억6712만
officialpsy [음악/댄스/가수] 1630만 91억7921만
BIGBANG [음악/댄스/가수] 1470만 71억7385만
TWICE [음악/댄스/가수] 1360만 33억0346만
Hongyu ASMR 홍유 [음식/요리/레시피] 1250만 37억8332만
TheWeekndVEVO [미분류] 1060만 144억7472만
Nao FunFun [키즈/어린이] 1060만 448만
핑크퐁 (인기 동요・동화) [키즈/어린이] 1050만 64억4642만
Stone Music Entertainment [음악/댄스/가수] 1040만 87억1231만
DuDuPopTOY [키즈/어린이] 1030만 47억1920만
Toymong tv 토이몽TV [키즈/어린이] 1000만 41억4286만
MBCkpop [TV/방송] 984만 80억1710만
Larva TUBA [키즈/어린이] 966만 42억5263만
[햄지]Hamzy [음식/요리/레시피] 946만 29억4059만
서은이야기[SeoeunStory] [키즈/어린이] 942만 32억

In [25]:
## 리스트에 이 모든 정보들 저장하기

results = [] # <- 정보를 저장할 빈 리스트 생성 
for channel in channel_list: 

    title = channel.select('h1 > a')[0].text.strip() 
    category = channel.select('p.category')[0].text.strip() 
    subscriber = channel.select('td.subscriber_cnt')[0].text.strip() 
    view = channel.select('td.view_cnt')[0].text.strip() 

    # 출력하기
    data = [title, category, subscriber, view]
    results.append(data)

In [26]:
results 

[['BLACKPINK', '[음악/댄스/가수]', '7390만', '235억0126만'],
 ['BANGTANTV', '[음악/댄스/가수]', '6590만', '159억7409만'],
 ['HYBE LABELS', '[음악/댄스/가수]', '6520만', '223억8194만'],
 ['SMTOWN', '[음악/댄스/가수]', '3030만', '244억0845만'],
 ['Boram Tube Vlog [보람튜브 브이로그]', '[키즈/어린이]', '2650만', '110억5288만'],
 ['1MILLION Dance Studio', '[음악/댄스/가수]', '2520만', '72억3740만'],
 ['JYP Entertainment', '[음악/댄스/가수]', '2440만', '163억1461만'],
 ['1theK (원더케이)', '[음악/댄스/가수]', '2400만', '221억3827만'],
 ['Mnet K-POP', '[음악/댄스/가수]', '1950만', '148억8730만'],
 ['KBS WORLD TV', '[TV/방송]', '1780만', '135억1256만'],
 ['JFlaMusic', '[음악/댄스/가수]', '1750만', '36억0286만'],
 ['Jane ASMR 제인', '[음식/요리/레시피]', '1640만', '61억6712만'],
 ['officialpsy', '[음악/댄스/가수]', '1630만', '91억7921만'],
 ['BIGBANG', '[음악/댄스/가수]', '1470만', '71억7385만'],
 ['TWICE', '[음악/댄스/가수]', '1360만', '33억0346만'],
 ['Hongyu ASMR 홍유', '[음식/요리/레시피]', '1250만', '37억8332만'],
 ['TheWeekndVEVO', '[미분류]', '1060만', '144억7472만'],
 ['Nao FunFun', '[키즈/어린이]', '1060만', '448만'],
 ['핑크퐁 (인기 동요・동화)', '[키즈/어린이]', '

In [27]:
# pandas를 활용하여 엑셀 파일에 저장하기
## 표 모양 DataFrame으로 바꿔주기

df = pd.DataFrame(results)
df

,0,1,2,3
0,BLACKPINK,[음악/댄스/가수],7390만,235억0126만
1,BANGTANTV,[음악/댄스/가수],6590만,159억7409만
2,HYBE LABELS,[음악/댄스/가수],6520만,223억8194만
3,SMTOWN,[음악/댄스/가수],3030만,244억0845만
4,Boram Tube Vlog [보람튜브 브이로그],[키즈/어린이],2650만,110억5288만
...,...,...,...,...
95,MayTree,[미분류],421만,7억8409만
96,로미유 스토리[Romiyu Story],[키즈/어린이],420만,11억1500만
97,[Dorothy]도로시,[음식/요리/레시피],417만,10억1293만
98,JaeYeol ASMR 재열,[취미/라이프],412만,11억7858만


In [28]:
df.columns = ['title', 'category', 'subscriber', 'view']
df

,title,category,subscriber,view
0,BLACKPINK,[음악/댄스/가수],7390만,235억0126만
1,BANGTANTV,[음악/댄스/가수],6590만,159억7409만
2,HYBE LABELS,[음악/댄스/가수],6520만,223억8194만
3,SMTOWN,[음악/댄스/가수],3030만,244억0845만
4,Boram Tube Vlog [보람튜브 브이로그],[키즈/어린이],2650만,110억5288만
...,...,...,...,...
95,MayTree,[미분류],421만,7억8409만
96,로미유 스토리[Romiyu Story],[키즈/어린이],420만,11억1500만
97,[Dorothy]도로시,[음식/요리/레시피],417만,10억1293만
98,JaeYeol ASMR 재열,[취미/라이프],412만,11억7858만


In [30]:
!pwd

/content


In [38]:
ls 

MyDrive/  Shareddrives/


In [39]:
cd /content/drive/MyDrive/Seminar/크롤링

/content/drive/MyDrive/Seminar/크롤링


In [40]:
# Dataframe을 엑셀로 저장하는 것은 정말 누워서 떡먹기
df.to_excel('/content/drive/MyDrive/Seminar/크롤링/youtube_rank_top100.xlsx', index=False)

In [41]:
ls

'1-1.Merge&Select&Sorting.ipynb'
 1-2.Edit_population.ipynb
 1-3.chicken_per_population.ipynb
 1.NaverShoping_Air_Analysis_Crawling.ipynb
'1.Selenium&HTML.ipynb'
 2.BeautifulSoup.ipynb
'[크롤링세션2]Google Play Store Review.ipynb'
 2.NaverShoping_Air_Analysis.ipynb
'Add) 10pages_crawling.ipynb'
'Add-1) NaverShopping_Airpot_Crawling(페이지자동전환).ipynb'
'[실습용]BeautifulSoup.ipynb'
'Pivot_table & Sort.ipynb'
 Youtube_Rank_Crawling.ipynb
'[크롤링세션]Youtube_Rank.ipynb'
 youtube_rank_top100.xlsx
